# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'averaged_perceptron_tagger', 'wordnet', 'stopwords'])
import json
import joblib
import numpy as np
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sqlalchemy import create_engine
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
def load_data(database = '../data/disaster-response-messages.db', sqltable= 'messages'):
    engine = create_engine('sqlite:///'+database)
    df = pd.read_sql_table(sqltable, con = engine)
    X = df['message']
    Y = df.iloc[:, 4:]
    return X,Y


In [5]:
X, y = load_data()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: http://sqlalche.me/e/e3q8)

In [28]:
df = pd.read_csv('message.csv')
X,y = df['message'], df.iloc[:, 5:]

In [29]:
y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [30]:
#define the regular expression for the URL
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
punctuations_list = string.punctuation

In [31]:
def tokenize(text):
            """
            Tokenize function
            
            Arguments:
                text -> list of text messages (Arabic)
            Output:
                clean_tokens -> tokenized text, clean for ML modeling
            """
            
            #replacing any url with "urlplaceholder" string
            detected_urls = re.findall(url_regex, text)
            for url in detected_urls:
                text = text.replace(url, "urlplaceholder")
                
            #normalize text
            text = text.lower()
            
            #tokenizing the text message
            tokens = word_tokenize(text)
            
            #defining the lemmatization object 
            lemmatizer = WordNetLemmatizer()
            
            #defining a translator object to remove all punctations
            table = str.maketrans('', '', punctuations_list)

            clean_tokens = []
            #cleaning every token by stemming and removing punctations and appending to the clean list
            for tok in tokens:
                clean_tok = lemmatizer.lemmatize(tok)     
                clean_tok = clean_tok.translate(table)        
                clean_tokens.append(clean_tok)
            
            #removing the stopwords from the clean_tokens list
            clean_tokens = [w for w in clean_tokens if w != '' and w not in stopwords.words('english')]


            return clean_tokens

In [32]:
#test tokenize function on the first five messages
for x in X[:5]:
    print(x)
    print(tokenize(x))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Is the Hurricane over or is it not over
['hurricane']
Looking for someone but no name
['looking', 'someone', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '8090', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [35]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x00000265CB756BF8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                      

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
Y_pred = pipeline.predict(X_test)


In [37]:

for i,col in enumerate(y_test.columns):
    print(col+'\n' ,classification_report(y_test.loc[:,col], Y_pred[:,i]))
        

related
               precision    recall  f1-score   support

           0       0.67      0.12      0.20      1563
           1       0.77      0.98      0.86      4944
           2       0.42      0.11      0.17        47

    accuracy                           0.77      6554
   macro avg       0.62      0.40      0.41      6554
weighted avg       0.75      0.77      0.70      6554

request
               precision    recall  f1-score   support

           0       0.90      0.97      0.94      5443
           1       0.77      0.50      0.61      1111

    accuracy                           0.89      6554
   macro avg       0.84      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

offer
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      

limit_output extension: Maximum message size of 10000 exceeded with 10826 characters

### 6. Improve your model
Use grid search to find better parameters. 

In [40]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 1.0),
        'vect__max_features': (None, 5000),
        'clf__estimator__n_estimators': [50, 100] ,
        'clf__estimator__learning_rate' : [0.03, 0.05]
}

cv = GridSearchCV(pipeline, param_grid=parameters)


In [41]:
cv.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                     

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [73]:
Y_pred = cv.predict(X_test)

#Test the model using accuracy, precision, recall
for i,col in enumerate(y_test.columns):
    accuracy = accuracy_score(y_test.iloc[:,i], Y_pred[:,i])
    precision = precision_score(y_test.iloc[:,i], Y_pred[:,i], average='micro')
    recall = recall_score(y_test.iloc[:,i], Y_pred[:,i], average='micro')
    print(col+'\n' , "accuracy: {}\tprecision: {}\trecall: {} \n".format(accuracy, precision, recall))

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [53]:
class UrlExtractor(BaseEstimator, TransformerMixin):
    """
    URL Extractor class
    
    This class extract the  'urlplaceholder' in the text,
    creating a new feature for the ML classifier
    """


    def finding_url(self, text):
        
        if 'urlplaceholder' in text:
            return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.finding_url)
        return pd.DataFrame(X_tagged)

In [54]:
class MessageLength(BaseEstimator, TransformerMixin):
    """
     Message Length Extractor class
    
    This class extract the message length ,
    creating a new feature for the ML classifier
    """

    def computing_message_length(self, text):
        return len(text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.computing_message_length)
        return pd.DataFrame(X_tagged)

In [62]:
class Tokenizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def tokenize(text):
            """
            Tokenize function
            
            Arguments:
                text -> list of text messages (Arabic)
            Output:
                clean_tokens -> tokenized text, clean for ML modeling
            """
            
            #replacing any url with "urlplaceholder" string
            detected_urls = re.findall(url_regex, text)
            for url in detected_urls:
                text = text.replace(url, "urlplaceholder")
                
            #normalize text
            text = text.lower()
            
            #tokenizing the text message
            tokens = word_tokenize(text)
            
            #defining the lemmatization object 
            lemmatizer = WordNetLemmatizer()
            
            #defining a translator object to remove all punctations
            table = str.maketrans('', '', punctuations_list)

            clean_tokens = []
            #cleaning every token by stemming and removing punctations and appending to the clean list
            for tok in tokens:
                clean_tok = lemmatizer.lemmatize(tok)     
                clean_tok = clean_tok.translate(table)        
                clean_tokens.append(clean_tok)
            
            #removing the stopwords from the clean_tokens list
            clean_tokens = [w for w in clean_tokens if w != '' and w not in stopwords.words('english')]

            return ' '.join(clean_tokens)
    
        return pd.Series(X).apply(tokenize).values

In [71]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('tokenize', Tokenizer()),
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer())
            ])),

            ('url_pipeline',Pipeline([
                ('tokenize', Tokenizer()),
                ('containing_url', UrlExtractor())
                ])),
            ('computing_message_length',MessageLength())       
            
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'clf__estimator__n_estimators': [50, 100,200] ,
        'features__transformer_weights': (
            {'text_pipeline': 1, 'url_pipeline' : 0.5, 'computing_message_length' : 0.5}, 
            {'text_pipeline': 0.5, 'url_pipeline' : 1,  'computing_message_length' : 1},
            {'text_pipeline': 0.8, 'url_pipeline' : 1,  'computing_message_length' : 1})
             
        
    
    }

cv = GridSearchCV(pipeline, param_grid=parameters)


In [ ]:
cv.fit(X_train, y_train)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
Y_pred = cv.predict(X_test)


In [ ]:
for i,col in enumerate(y_test.columns):
    
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(Y_test.iloc[:,i], y_pred[:,i], labels=labels)
    accuracy = (y_pred[:,i] == Y_test.iloc[:,i]).mean()
    class_report = classification_report(Y_test.iloc[:,i], y_pred[:,i])
    f1_sc = f1_score(Y_test.iloc[:,i], y_pred[:,i])
    prec = precision_score(Y_test.iloc[:,i], y_pred[:,i])

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy: ", accuracy)
    print("precision: ", prec)
    print("f1_score: ", f1_sc)
    print("\nClassification report:\n ", class_report ) 

print("\nBest Parameters: ", cv.best_params_)


### 9. Export your model as a pickle file

In [ ]:
joblib.dump(cv, './models/clf.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.